# ROI + Oversampling Validation (Sector 20 / Camera 3 / CCD 3)

This notebook executes two sections:
1. ROI-only downsampling (no oversampling) on a 100x100 ROI.
2. Oversampling-enabled downsampling on the same ROI, then block-downsample and compare against section 1.

All tests use `single-offset` mode (`dx=0, dy=0`) for speed.

In [1]:
from pathlib import Path
import subprocess
import numpy as np
from astropy.io import fits

repo = Path('/home/kshukawa/syndiff')
script = repo / 'multi_offset_downsampling.py'

sector, camera, ccd = 20, 3, 3
x_min, y_min, x_max, y_max = 1000, 1000, 1010, 1010
roi_h, roi_w = y_max - y_min, x_max - x_min

baseline_dir = repo / 'data/shifted_downsampled/sector0020_camera3_ccd3'

## Section A: ROI feature test (no oversampling)

In [2]:
cmd_roi = [
    'bash', '-lc',
    'source ~/.bashrc && mamba activate syndiff && ' +
    f'python {script} {sector} {camera} {ccd} ' +
    f'--x-min {x_min} --y-min {y_min} --x-max {x_max} --y-max {y_max} ' +
    f'--single-offset'
]

print('Running ROI-only command...')
res = subprocess.run(cmd_roi, cwd=repo, capture_output=True, text=True)
print(res.stdout[-2000:])
if res.returncode != 0:
    print(res.stderr)
    raise RuntimeError(f'ROI-only run failed: {res.returncode}')
print('ROI-only run completed successfully.')

Running ROI-only command...


Loading TESS data and WCS...
Using ROI (base TESS scale): x=[1000,1010), y=[1000,1010)
Loading skycell info...
Prefiltered to 1 ROI-intersecting skycells
Loading Zarr metadata...
Precomputing shifts for all offsets...
Getting registration files...
Processing 1 skycells in 1 batches...
Completed batch 1
Combining results...
Saving outputs...
Done! Total processing time: 0.14 minutes
Processing completed at: Sat Feb 14 00:28:22 2026
Total time: 0.14 minutes
Processed 1 skycells in 1 batches
Generated 1 shifted images
Ignored mask bits: [12]
Shifts processed:
  dx=0.000, dy=0.000
Results saved to: /home/kshukawa/syndiff/data/shifted_downsampled/sector0020_camera3_ccd3_x1000-1010_y1000-1010

ROI-only run completed successfully.


In [3]:
baseline_dir

PosixPath('/home/kshukawa/syndiff/data/shifted_downsampled/sector0020_camera3_ccd3')

In [8]:
baseline_file = baseline_dir / 'syndiff_template_s0020_3_3_dx0.000_dy0.000.fits'
roi_file = repo / 'data/shifted_downsampled/sector0020_camera3_ccd3_x1000-1010_y1000-1010/syndiff_template_s0020_3_3_x1000-1010_y1000-1010_dx0.000_dy0.000.fits'

assert baseline_file.exists(), f'Missing baseline file: {baseline_file}'
assert roi_file.exists(), f'Missing ROI output file: {roi_file}'

with fits.open(baseline_file) as hb:
    base_flux = hb['FLUX_SUM'].data.astype(np.float32)
    base_count = hb['COUNT'].data.astype(np.float32)
    base_mask = hb['MASK'].data.astype(np.float32)

with fits.open(roi_file) as hr:
    roi_flux = hr['FLUX_SUM'].data.astype(np.float32)
    roi_count = hr['COUNT'].data.astype(np.float32)
    roi_mask = hr['MASK'].data.astype(np.float32)
    hdr = hr['FLUX_SUM'].header

base_flux_roi = base_flux[y_min:y_max, x_min:x_max]
base_count_roi = base_count[y_min:y_max, x_min:x_max]
base_mask_roi = base_mask[y_min:y_max, x_min:x_max]

assert roi_flux.shape == (roi_h, roi_w), f'Unexpected ROI flux shape: {roi_flux.shape}'
assert roi_count.shape == (roi_h, roi_w), f'Unexpected ROI count shape: {roi_count.shape}'
assert roi_mask.shape == (roi_h, roi_w), f'Unexpected ROI mask shape: {roi_mask.shape}'

assert np.allclose(roi_flux, base_flux_roi, rtol=1e-6, atol=1e-6), 'ROI FLUX mismatch vs baseline crop'
assert np.allclose(roi_count, base_count_roi, rtol=1e-6, atol=1e-6), 'ROI COUNT mismatch vs baseline crop'
assert np.allclose(roi_mask, base_mask_roi, rtol=1e-6, atol=1e-6), 'ROI MASK mismatch vs baseline crop'

print('Section A PASS: ROI-only output matches baseline crop.')
print('Header ROI keys:', {k: hdr[k] for k in ['XMIN','XMAX','YMIN','YMAX','ROIW','ROIH'] if k in hdr})

Section A PASS: ROI-only output matches baseline crop.
Header ROI keys: {'XMIN': 1000, 'XMAX': 1010, 'YMIN': 1000, 'YMAX': 1010, 'ROIW': 10, 'ROIH': 10}


## Section B: Oversampling support test

In [9]:
cmd_os = [
    'bash', '-lc',
    'source ~/.bashrc && mamba activate syndiff && ' +
    f'python {script} {sector} {camera} {ccd} ' +
    f'--x-min {x_min} --y-min {y_min} --x-max {x_max} --y-max {y_max} ' +
    f'--oversampling-factor 2 --single-offset'
]

print('Running oversampling command...')
res_os = subprocess.run(cmd_os, cwd=repo, capture_output=True, text=True)
print(res_os.stdout[-2000:])
if res_os.returncode != 0:
    print(res_os.stderr)
    raise RuntimeError(f'Oversampling run failed: {res_os.returncode}')
print('Oversampling run completed successfully.')

Running oversampling command...
Loading TESS data and WCS...
Using ROI (base TESS scale): x=[1000,1010), y=[1000,1010)
Loading skycell info...
Prefiltered to 1 ROI-intersecting skycells
Loading Zarr metadata...
Precomputing shifts for all offsets...
Getting registration files...
Processing 1 skycells in 1 batches...
Completed batch 1
Combining results...
Saving outputs...
Done! Total processing time: 0.14 minutes
Processing completed at: Sat Feb 14 00:30:38 2026
Total time: 0.14 minutes
Processed 1 skycells in 1 batches
Generated 1 shifted images
Ignored mask bits: [12]
Shifts processed:
  dx=0.000, dy=0.000
Results saved to: /home/kshukawa/syndiff/data/shifted_downsampled/sector0020_camera3_ccd3_x1000-1010_y1000-1010_os2

Oversampling run completed successfully.


In [11]:
os_file = repo / 'data/shifted_downsampled/sector0020_camera3_ccd3_x1000-1010_y1000-1010_os2/syndiff_template_s0020_3_3_x1000-1010_y1000-1010_os2_dx0.000_dy0.000.fits'

assert os_file.exists(), f'Missing oversampling output file: {os_file}'

with fits.open(os_file) as ho:
    os_flux = ho['FLUX_SUM'].data.astype(np.float32)
    os_count = ho['COUNT'].data.astype(np.float32)
    os_mask = ho['MASK'].data.astype(np.float32)
    os_hdr = ho['FLUX_SUM'].header

assert os_flux.shape == (roi_h * 2, roi_w * 2), f'Unexpected OS flux shape: {os_flux.shape}'
assert os_count.shape == (roi_h * 2, roi_w * 2), f'Unexpected OS count shape: {os_count.shape}'
assert os_mask.shape == (roi_h * 2, roi_w * 2), f'Unexpected OS mask shape: {os_mask.shape}'

def block_sum_2x2(arr):
    h, w = arr.shape
    assert h % 2 == 0 and w % 2 == 0
    return arr.reshape(h // 2, 2, w // 2, 2).sum(axis=(1, 3))

os_flux_ds = block_sum_2x2(os_flux)
os_count_ds = block_sum_2x2(os_count)
os_mask_ds = block_sum_2x2(os_mask)

# Compare against Section A ROI output
with fits.open(roi_file) as hr:
    roi_flux_ref = hr['FLUX_SUM'].data.astype(np.float32)
    roi_count_ref = hr['COUNT'].data.astype(np.float32)
    roi_mask_ref = hr['MASK'].data.astype(np.float32)

flux_diff = np.abs(os_flux_ds - roi_flux_ref)
count_diff = np.abs(os_count_ds - roi_count_ref)
mask_diff = np.abs(os_mask_ds - roi_mask_ref)

print('Oversampling header keys:', {k: os_hdr[k] for k in ['OVERSAMP','XMIN','XMAX','YMIN','YMAX','ROIW','ROIH'] if k in os_hdr})
print(f'Flux diff: max={flux_diff.max():.6f}, mean={flux_diff.mean():.6f}')
print(f'Count diff: max={count_diff.max():.6f}, mean={count_diff.mean():.6f}')
print(f'Mask diff: max={mask_diff.max():.6f}, mean={mask_diff.mean():.6f}')

# Near-identical check after downsampling; allow small numeric differences
assert np.allclose(os_flux_ds, roi_flux_ref, rtol=2e-2, atol=1e-2), 'Oversampled->downsampled FLUX not near ROI reference'
assert np.allclose(os_count_ds, roi_count_ref, rtol=2e-2, atol=1e-2), 'Oversampled->downsampled COUNT not near ROI reference'
assert np.allclose(os_mask_ds, roi_mask_ref, rtol=2e-2, atol=1e-2), 'Oversampled->downsampled MASK not near ROI reference'

print('Section B PASS: Oversampled output downsampled back is near-identical to ROI-only result.')

Oversampling header keys: {'OVERSAMP': 2, 'XMIN': 1000, 'XMAX': 1010, 'YMIN': 1000, 'YMAX': 1010, 'ROIW': 10, 'ROIH': 10}
Flux diff: max=0.000977, mean=0.000022
Count diff: max=0.000000, mean=0.000000
Mask diff: max=0.000000, mean=0.000000
Section B PASS: Oversampled output downsampled back is near-identical to ROI-only result.


## Final status
If all assertions above passed, both Section A and Section B are complete.